In [1]:
import os
import shutil
import pandas as pd

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt

# 디렉터리로 클래스 분류

In [2]:
train_df = pd.read_csv('./dataset/train_df.csv')
train_df.head()

,index,file_name,class,state,label
0,0,10000.png,transistor,good,transistor-good
1,1,10001.png,capsule,good,capsule-good
2,2,10002.png,transistor,good,transistor-good
3,3,10003.png,wood,good,wood-good
4,4,10004.png,bottle,good,bottle-good


In [3]:
train = train_df["file_name"].to_numpy()
target = train_df["label"].to_numpy()

In [4]:
train_path = "./dataset/main_directory/"

if not os.path.exists(train_path):
        os.mkdir(train_path)

for label in train_df["label"].unique():
    if not os.path.exists(train_path+label):
        os.mkdir(train_path+label)

In [5]:
for file, label in zip(train,target):
    src = "./dataset/train/"+file
    dst = train_path + label + '/' + label + '_train_' + file
    shutil.copy(src, dst)

# 데이터 불러오기

In [6]:
INPUT_SHAPE = (300,300,3)
BATCH_SIZE = 4
AUTOTUNE = tf.data.AUTOTUNE

In [7]:
test_path = "./dataset/test/"

train_ds = keras.preprocessing.image_dataset_from_directory(
    directory = train_path,
    labels="inferred",
    label_mode="categorical",
    batch_size=BATCH_SIZE,
    image_size=INPUT_SHAPE[:2],
    shuffle=True,
    subset="training",
    seed = 0,
    validation_split=0.2,
)

valid_ds = keras.preprocessing.image_dataset_from_directory(
    directory = train_path,
    labels="inferred",
    label_mode="categorical",
    batch_size=BATCH_SIZE,
    image_size=INPUT_SHAPE[:2],
    shuffle=True,
    subset="validation",
    seed = 0,
    validation_split=0.2,
)

test_ds = keras.preprocessing.image_dataset_from_directory(
    directory = test_path,
    labels=None,
    batch_size=BATCH_SIZE,
    image_size=INPUT_SHAPE[:2],
    shuffle=False,
)

Found 4277 files belonging to 88 classes.
Using 3422 files for training.
Found 4277 files belonging to 88 classes.
Using 855 files for validation.
Found 2154 files belonging to 1 classes.


# 데이터 전처리 층만들기
## evaluate() 또는 predict() 호출 시에는 자동으로 비활성화 된다.

In [8]:
data_augmentation = keras.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    keras.layers.experimental.preprocessing.RandomRotation(0.2),
    keras.layers.experimental.preprocessing.RandomTranslation(0.1,0.1),
])

In [14]:
transfer_model = keras.applications.MobileNet(
    input_shape= INPUT_SHAPE,
    include_top= False,
    weights= 'imagenet',
)
transfer_model.trainable = False
transfer_model.summary()

17235968/17225924 [==============================] - 1s 0us/step
Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 150, 150, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 150, 150, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 150, 150, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 150, 150, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 150, 150, 32

 conv_dw_8_bn (BatchNormaliz  (None, 18, 18, 512)      2048      
 ation)                                                          
                                                                 
 conv_dw_8_relu (ReLU)       (None, 18, 18, 512)       0         
                                                                 
 conv_pw_8 (Conv2D)          (None, 18, 18, 512)       262144    
                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 18, 18, 512)      2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 18, 18, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 18, 18, 512)      4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 18, 18, 512)      2048      
 ation)   

In [15]:
finetune_model = keras.Sequential([
    keras.Input(shape=INPUT_SHAPE),
#     data_augmentation,
    keras.layers.experimental.preprocessing.Rescaling(1./255),
    transfer_model,
    keras.layers.Flatten(),
    keras.layers.Dense(200, activation='relu'),
    keras.layers.Dense(88, activation='softmax')
],"my_model")
finetune_model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 300, 300, 3)       0         
                                                                 
 mobilenet_1.00_224 (Functio  (None, 9, 9, 1024)       3228864   
 nal)                                                            
                                                                 
 flatten (Flatten)           (None, 82944)             0         
                                                                 
 dense (Dense)               (None, 200)               16589000  
                                                                 
 dense_1 (Dense)             (None, 88)                17688     
                                                                 
Total params: 19,835,552
Trainable params: 16,606,688
Non-trainable params: 3,228,864
______________________________________

In [16]:
finetune_model.compile(loss='categorical_crossentropy',
                      optimizer=keras.optimizers.Adam(learning_rate=0.0002),
                       metrics='accuracy')                    

In [ ]:
augmented_train_ds = train_ds.map(
    lambda x,y:(data_augmentation(x,training=True),y)
)
augmented_valid_ds = valid_ds.map(
    lambda x,y:(data_augmentation(x,training=True),y)
)

hist = finetune_model.fit(
    train_ds.cache().prefetch(buffer_size=AUTOTUNE),
    validation_data=valid_ds.cache().prefetch(buffer_size=AUTOTUNE),
    epochs=2
)

Epoch 1/2


In [ ]:
class_names = train_ds.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")